#### Library and image loading

In [1]:
# Installing mask model
!pip install -qr https://huggingface.co/briaai/RMBG-1.4/resolve/main/requirements.txt

In [ ]:
# Removing background
from transformers import pipeline
image_path = "500434410-800x600.webp"
pipe = pipeline("image-segmentation", model="briaai/RMBG-1.4", trust_remote_code=True)
pillow_mask = pipe(image_path, return_mask = True) # outputs a pillow mask
pillow_image = pipe(image_path)
pillow_image.save("pillow_image.png")

In [6]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(pillow_image)

#### Creating mask_image

In [21]:
from PIL import Image

# Extract the alpha channel (transparency)
alpha = pillow_image.getchannel("A")

# Create a binary mask from the alpha channel
mask = alpha.point(lambda p: 0 if p > 0 else 255)  # White for subject, black for background

# Save and display the mask image
mask.save("mask_image.png")
mask.show()

#### Generating background

In [ ]:
from PIL import Image
from diffusers import StableDiffusionInpaintPipeline

# Load the pipeline
#Takes a lot the first time
pipe = StableDiffusionInpaintPipeline.from_pretrained("runwayml/stable-diffusion-inpainting")
#pipe = pipe.to("cuda")  # Use "cpu" if GPU is not available uncomment this if you have CUDA gpu power

# Load the input image and mask
image = Image.open("pillow_image.png").convert("RGB")
mask = Image.open("mask_image.png").convert("RGB")

# Ensure the mask and image are the same size
if image.size != mask.size:
    mask = mask.resize(image.size, Image.NEAREST)

# Perform inpainting
result = pipe(prompt="Huge palace on bel air", image=image, mask_image=mask).images[0]

# Save and display the result
result.save("output.jpg")
result.show()